# 데이터 탐색

100개의 RFP 문서와 메타데이터를 탐색합니다.

In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from collections import Counter

# 프로젝트 루트 경로 추가
project_root = Path().absolute().parent
sys.path.append(str(project_root))

from src import config

print(f"프로젝트 루트: {project_root}")
print(f"데이터 디렉토리: {config.RAW_DATA_DIR}")

## 1. 메타데이터 CSV 파일 로드

In [ ]:
# CSV 파일 경로
csv_path = config.RAW_DATA_DIR / "data_list.csv"

if not csv_path.exists():
    print("❌ data_list.csv 파일이 없습니다!")
    print("   Google Drive에서 다운로드하거나 수동으로 배치해주세요.")
else:
    # CSV 로드 (인코딩 자동 감지)
    try:
        df = pd.read_csv(csv_path, encoding='utf-8')
    except:
        df = pd.read_csv(csv_path, encoding='cp949')
    
    print(f"✅ 메타데이터 로드 완료: {len(df)}개 문서")
    print(f"\n컬럼 목록:")
    for col in df.columns:
        print(f"  - {col}")

In [ ]:
# 데이터 미리보기
df.head()

In [ ]:
# 데이터 기본 정보
df.info()

## 2. RFP 문서 파일 확인

In [ ]:
# data/raw 디렉토리의 파일 목록
rfp_files = list(config.RAW_DATA_DIR.glob("*"))
rfp_files = [f for f in rfp_files if f.is_file() and f.suffix.lower() in ['.pdf', '.hwp']]

print(f"총 RFP 문서 파일 수: {len(rfp_files)}개")

# 파일 포맷별 개수
file_extensions = [f.suffix.lower() for f in rfp_files]
ext_counts = Counter(file_extensions)

print("\n파일 포맷별 개수:")
for ext, count in ext_counts.items():
    print(f"  {ext}: {count}개")

In [ ]:
# 파일 크기 분포
file_sizes = [f.stat().st_size / 1024 / 1024 for f in rfp_files]  # MB 단위

print("파일 크기 통계 (MB):")
print(f"  평균: {np.mean(file_sizes):.2f}")
print(f"  중앙값: {np.median(file_sizes):.2f}")
print(f"  최소: {np.min(file_sizes):.2f}")
print(f"  최대: {np.max(file_sizes):.2f}")
print(f"  총합: {np.sum(file_sizes):.2f}")

In [ ]:
# 파일명 샘플 출력
print("\n파일명 샘플 (처음 10개):")
for i, f in enumerate(rfp_files[:10], 1):
    size = f.stat().st_size / 1024 / 1024
    print(f"{i:2d}. {f.name} ({size:.2f} MB)")

## 3. 메타데이터와 파일명 매칭 확인

In [ ]:
# CSV에 파일명 컬럼이 있는지 확인하고 매칭
if 'file_name' in df.columns or '파일명' in df.columns:
    filename_col = 'file_name' if 'file_name' in df.columns else '파일명'
    
    csv_filenames = set(df[filename_col].str.lower())
    actual_filenames = set(f.name.lower() for f in rfp_files)
    
    # 매칭 확인
    matched = csv_filenames & actual_filenames
    only_in_csv = csv_filenames - actual_filenames
    only_in_files = actual_filenames - csv_filenames
    
    print(f"매칭된 파일: {len(matched)}개")
    print(f"CSV에만 있는 파일: {len(only_in_csv)}개")
    print(f"디렉토리에만 있는 파일: {len(only_in_files)}개")
    
    if only_in_csv:
        print(f"\n⚠️ CSV에는 있지만 파일이 없는 경우:")
        for fn in list(only_in_csv)[:5]:
            print(f"  - {fn}")
    
    if only_in_files:
        print(f"\n⚠️ 파일은 있지만 CSV에 없는 경우:")
        for fn in list(only_in_files)[:5]:
            print(f"  - {fn}")
else:
    print("⚠️ 메타데이터에 파일명 컬럼이 없습니다.")
    print(f"   사용 가능한 컬럼: {list(df.columns)}")

## 4. 메타데이터 분석

In [ ]:
# 주요 메타데이터 분석
print("메타데이터 분석:")
print("="*50)

# 발주기관 분포 (컬럼명은 실제 CSV에 맞춰 수정)
if '발주기관' in df.columns or 'agency' in df.columns:
    agency_col = '발주기관' if '발주기관' in df.columns else 'agency'
    print(f"\n발주기관 종류: {df[agency_col].nunique()}개")
    print("\nTop 10 발주기관:")
    print(df[agency_col].value_counts().head(10))

# 예산 분포
if '예산' in df.columns or 'budget' in df.columns:
    budget_col = '예산' if '예산' in df.columns else 'budget'
    print(f"\n예산 통계:")
    print(df[budget_col].describe())

# 사업명 길이 분포
if '사업명' in df.columns or 'project_name' in df.columns:
    project_col = '사업명' if '사업명' in df.columns else 'project_name'
    df['사업명_길이'] = df[project_col].str.len()
    print(f"\n사업명 길이 통계:")
    print(df['사업명_길이'].describe())

In [ ]:
# 결측치 확인
print("\n결측치 개수:")
print(df.isnull().sum())

## 5. 데이터 준비 상태 요약

In [ ]:
print("\n" + "="*60)
print("데이터 준비 상태 요약")
print("="*60)

print(f"\n✅ 메타데이터: {len(df)}개 레코드")
print(f"✅ RFP 문서: {len(rfp_files)}개 파일")
print(f"   - PDF: {ext_counts.get('.pdf', 0)}개")
print(f"   - HWP: {ext_counts.get('.hwp', 0)}개")
print(f"\n총 데이터 크기: {sum(file_sizes):.2f} MB")

print("\n다음 단계: 02_document_loading.ipynb")

## 6. 샘플 문서 선택

테스트용으로 사용할 샘플 문서를 선택합니다.

In [ ]:
# PDF와 HWP 각각 1개씩 샘플 선택
pdf_files = [f for f in rfp_files if f.suffix.lower() == '.pdf']
hwp_files = [f for f in rfp_files if f.suffix.lower() == '.hwp']

if pdf_files:
    sample_pdf = pdf_files[0]
    print(f"샘플 PDF: {sample_pdf.name}")

if hwp_files:
    sample_hwp = hwp_files[0]
    print(f"샘플 HWP: {sample_hwp.name}")

# 샘플 파일 경로 저장 (다음 노트북에서 사용)
%store sample_pdf
%store sample_hwp